**Estadística vs. Causal**: Incluso con una cantidad infinita de datos, a veces no podemos calcular algunas cantidades causales. Por el contrario, gran parte de las estadísticas se trata de abordar la incertidumbre en muestras finitas. Cuando se dan datos infinitos, no hay incertidumbre. Sin embargo, la asociación, un concepto estadístico, no es causalidad. Hay más trabajo por hacer en la inferencia causal, incluso después de comenzar con datos infinitos. Esta es la distinción principal que motiva la inferencia causal.

**Identificación frente a estimación**: la identificación de los efectos causales es exclusiva de la inferencia causal. Es el problema que queda por resolver, incluso cuando tenemos datos infinitos. Sin embargo, la inferencia causal también comparte estimación con la estadística tradicional y el aprendizaje automático.

**Intervencional vs Observacional**: Si podemos intervenir/experimentar,
la identificación de los efectos causales es relativamente fácil. Esto se debe simplemente a que podemos tomar la acción cuyo efecto causal queremos medir y simplemente medir el efecto después de tomar esa acción. Los datos de observación es donde se vuelve más complicado porque casi siempre se introduce confusión (ruido) en los datos.

¿Por qué nuestras mentes estarían tan "cableadas" para pensar en relaciones causales por encima de las meramente correlacionales? Una buena razón sería que _las correlaciones que son causales son más estables en el tiempo y ante cambios en el entorno_. En palabras de Pearl, traducidas de su importante libro _Causality_,

> ... las relaciones causales son _ontológicas_, al describir vínculos físicos objetivos de nuestro mundo, mientras que las relaciones probabilísticas son _epistémicas_, reflejando lo que sabemos o creemos sobre el mundo.

Tiene sentido pues que la capacidad de _generalizar_ de un modelo de inteligencia artificial y la capacidad de _transferir_ el aprendizaje a una tarea nueva dependan de la capacidad del modelo de aislar los _mecanismos causales_ correctos (o más probables) a partir de los datos. Sin proponer que algunas correlaciones corresponden a mecanismos causales, mientras que otras no, es imposible saber qué correlaciones se mantendrían y cuáles serían destruidas al modificarse algún factor del entorno generador de los datos.

Actualmente hay muchísimo interés en las posibilidades de combinar o aumentar las capacidades de los modelos de Machine Learning con métodos de inferencia causal

### Causalidad no es correlación

(Mejor dicho, _correlación no implica causalidad_.)
Las _correlaciones espurias_ nos muestran que dos variables pueden aparecer asociadas, es decir que "se mueven conjuntamente", en este caso en el tiempo, incluso cuando lo más plausible es que no tengan absolutamente nada que ver.

También pueden utilizarse para enfatizar que la correlación presente en los datos puede ser altamente _no lineal_, por lo cual una medida como la correlación de Pearson, que mide correlación _lineal_, puede resultar engañosa. Pero hay medidas de correlación más fidedignas en el caso no lineal (como por ejemplo, la _entropía relativa_, también llamada _divergencia de Kullback-Leibler_).

Detrás de todas estas nociones de correlación (o "asociación", como se dice a veces para distinguir de la correlación puramente lineal) hay un concepto fundamental, que es el de la **dependencia probabilística**. Dadas dos variables aleatorias $X, Y$, decimos que son **probabilísticamente independientes** si su distribución de probabilidad conjunta se factoriza como $p(X, Y) = p(X) p(Y)$. Equivalentemente, condicionar sobre una de las dos variables no afecta la distribución de valores de la otra:

$$ p(X | Y=y) = p(X) $$
$$ p(Y | X=x) = p(Y) $$

Vamos a usar una notación especial para esta situación de independencia entre $X$ e $Y$: 

$$\newcommand{\indep}{\perp \!\!\! \perp} X \indep Y$$

También usaremos fuertemente el concepto de **independencia condicional**: $X$ es independiente de $Y$ _dado_ $Z$ (escribimos $X \indep Y | Z$) si $p(X, Y | Z) = p(X | Z) p(Y | Z)$. Volveremos sobre esto en la Clase 2, pero por ahora es importante que **cuando hablamos de correlación, no nos referimos solamente a la correlación lineal** sino que **dos variables $X$, $Y$ están correlacionadas en alguna medida siempre que $p(X, Y) \neq p(X) p(Y)$**.

---
Dijimos que correlación no implica causalidad, pero por el contrario, **si veo correlación, es más probable que haya causalidad que si no observo nada**. De hecho el filósofo Hans Reichenbach formuló a mediados del Siglo XX el [_Principio de la Causa Común_](https://plato.stanford.edu/entries/physics-Rpcc/), que lleva su nombre, según el cual **dadas dos variables correlacionadas, o bien una es causa de la otra, o bien al revés, o bien ambas tienen una causa común**. Es decir que **la correlación es nuestra herramienta para detectar causalidad, y trataremos de cómo _identificar_ cuándo correlación _sí_ implica causalidad.**

Pero entonces ¿Qué podemos decir sobre lo que _sí_ es la causalidad?


---

# Parte II: Marco conceptual: resultados potenciales (o _potential outcomes_)

Este marco trata de responder a la pregunta de si "A" causa "B" a partir de un contrafáctico: comparar "B dado A" con "B dado no A". La pregunta usual que uno se suele hacer.

Vamos a introducir algunos conceptos que son parte del modelo causal de Neyman-Rubin. 

In [ ]:
#@title {display-mode: "form"}
from graphviz import Digraph
from IPython.display import display

#Especificamos engine='neato' para poder usar argumento `pos` en nodos
dot = Digraph(engine='neato')
dot.node('T', 'Tratamiento', pos='1,1!')
dot.node('Y', 'Resultado', pos='3,1!')
dot.node('X', 'Tamaño de los cálculos', pos='2,2!')
dot.edges(['TY', 'XY', 'TX'])
dot

In [ ]:
#@title {display-mode: "form"} 
from graphviz import Digraph
from IPython.display import display

#Especificamos engine='neato' para poder usar argumento `pos` en nodos
dot = Digraph(engine='neato')
dot.node('T', 'Hospital', pos='1,1!')
dot.node('Y', 'Estado de salud', pos='3,1!')
dot.node('X', 'Condición de salud previa', pos='2,2!')
dot.edges(['TY', 'XY', 'XT'])
dot

### Confusores y sesgos 

* Decimos que la "condición de salud" previa es un **confusor**. 
  * Una variable que está correlacionada tanto con el tratamiento $T_i$ (ir al hospital) como con el resultado $Y_i$ (el estado de salud posterior)
  * Si no la tomamos en cuenta, confundimos el efecto que queremos medir, la visita al hospital (la atención médica y otros servicios recibidos allí) con el confusor
* Decimos que cuando comparamos la salud promedio observada entre quienes fueron al hospital y quienes no, además del **efecto causal** de la visita al hospital, tenemos un **sesgo de selección**: las personas que "se autoseleccionan" para ir a hospital son distintas de las que no van. 
* **No** estamos cumpliendo con la comparación **_ceteris paribus_**: las circunstancias de un grupo (condición mala de salud previa), no son iguales a las del otro (condiciones buenas de salud previa)

* Comparar al grupo de personas que va al hospital con el grupo de personas que no va no sirve para responder nuestra pregunta causal, contrafáctica: ¿una persona que sí fue, está mejor que si no hubiera ido? (es decir, nuestra pregunta por el ITE, o por el ATE)

* Entonces, el grupo de personas que no va al hospital **no es un buen contrafáctico** de las personas que sí van al hospital


Como tenemos el DeLorean, podemos **observar y comparar ambos resultados potenciales**. El efecto individual de tratamiento (**ITE**, por _individual treatment effect_) para Marty será la diferencia en el resultado de salud cuando Marty va al hospital y el resultado de salud cuando Marty no va al hospital:


\begin{equation}
 ITE_{Marty} =  \underbrace{Y_{Marty}(T_{Marty} = 1)}_{\substack{\text{ Resultado observado} \\ \text{dado que fue al hospital}}}
- \underbrace{Y_{Marty}(T_{Marty} = 0)}_{\substack{\text{ Resultado observado} \\ \text{dado que NO fue al hospital}}} = Y(1) - Y(0)
\end{equation}

En particular, a la media de los ITE la llamamos efecto medio de tratamiento (**ATE**, por _Average Treatment Effect_). 

_La diferencia de medias entre los resultados observados de grupos tratados y no tratados a veces se la llama "diferencia asociacional"_

* ATE: $\E[Y(1) - Y(0)] = \E[Y(1)] - \E[Y(0)]$ 
* Diferencia asociacional: $\E[(Y|T=1)] - \E[(Y|T=0)]$

**Ejercicio de lectura activa**. ¿En qué se diferencian el ATE y la diferencia asociacional?

\lim\limits_{n\rightarrow\infty}{\left(1+\frac{1}{n}\right)^n
\lim\limits_{n\rightarrow\infty}{\left(1+\frac{1}{n}\right)^n}